In [5]:
import fxcmpy
import pandas as pd
import numpy as np
import datetime as dt

from pyti.accumulation_distribution import accumulation_distribution as ad
from pyti.aroon import aroon_up
from pyti.aroon import aroon_down
from pyti.average_true_range import average_true_range as atr
from pyti.chande_momentum_oscillator import chande_momentum_oscillator as cmo
from pyti.chaikin_money_flow import chaikin_money_flow as cmf 
from pyti.commodity_channel_index import commodity_channel_index as cci
from pyti.exponential_moving_average import exponential_moving_average as ema
from pyti.hull_moving_average import hull_moving_average as hma
from pyti.money_flow_index import money_flow_index as mfi
from pyti.on_balance_volume import on_balance_volume as obv
from pyti.simple_moving_average import simple_moving_average as sma
from pyti.stochastic import percent_k as percent_k
from pyti.stochastic import percent_d as percent_d
from pyti.smoothed_moving_average import smoothed_moving_average as smoothed_ma
from pyti.true_range import true_range as tr
from pyti.ultimate_oscillator import ultimate_oscillator as uo
from pyti.volatility import volatility as volat
from pyti.relative_strength_index import relative_strength_index as rsi
from pyti.williams_percent_r import williams_percent_r as wpr 

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import roc_auc_score 

In [6]:
con = fxcmpy.fxcmpy(config_file='fxcm.cfg')
df = con.get_candles('GBP/JPY', period='D1',start= dt.datetime(2012, 1, 1),end = dt.datetime(2018, 7, 13))


,date,bidopen,bidclose,bidhigh,bidlow,askopen,askclose,askhigh,asklow,tickqty,...,percent_k,percent_d,smoothed_ma,true_range,ulti_osc,volatility,rsi,williams,trend,trend_nextday
0,2012-01-02 22:00:00,119.526,119.242,119.527,118.955,119.618,119.318,119.622,119.025,12277,...,NaN,NaN,119.318000,-97.977436,NaN,NaN,NaN,-97.977436,1,0.0
1,2012-01-03 22:00:00,119.242,120.033,120.161,119.154,119.318,120.091,120.194,119.233,52708,...,NaN,NaN,119.724842,-96.984143,NaN,NaN,NaN,-96.984143,0,0.0
2,2012-01-04 22:00:00,120.033,119.799,120.196,119.599,120.091,119.849,120.251,119.622,52090,...,NaN,NaN,119.770657,-97.295109,NaN,NaN,NaN,-97.295109,0,0.0
3,2012-01-05 22:00:00,119.799,119.451,119.949,119.169,119.849,119.505,119.977,119.196,63691,...,NaN,NaN,119.693409,-97.737144,NaN,NaN,NaN,-97.737144,0,1.0
4,2012-01-06 22:00:00,119.451,118.714,119.734,118.540,119.505,118.780,119.763,118.566,53869,...,NaN,NaN,119.470359,-98.668757,NaN,NaN,NaN,-98.668757,1,1.0
5,2012-01-09 22:00:00,118.714,118.702,118.866,118.254,118.780,118.903,118.903,118.289,48787,...,NaN,NaN,119.349273,-98.510704,NaN,NaN,NaN,-98.510704,1,0.0
6,2012-01-10 22:00:00,118.702,118.961,119.063,118.697,118.903,119.023,119.088,118.744,53127,...,NaN,NaN,119.286733,-98.356506,NaN,NaN,NaN,-98.356506,0,0.0
7,2012-01-11 22:00:00,118.961,117.789,119.109,117.711,119.023,117.841,119.133,117.739,56479,...,NaN,NaN,119.032888,-99.875357,NaN,NaN,NaN,-99.875357,0,1.0
8,2012-01-12 22:00:00,117.789,117.662,118.108,117.512,117.841,117.750,118.131,117.535,62647,...,NaN,NaN,118.823464,-99.992290,NaN,NaN,NaN,-99.992290,1,0.0
9,2012-01-13 22:00:00,117.662,117.809,118.186,117.275,117.750,118.006,118.211,117.295,68173,...,0.109355,NaN,118.697955,-99.663334,NaN,1.211476,NaN,-99.663334,0,1.0
